In [1]:
# ==============================================================================
# NOTEBOOK 04: ETL de Dados de Satélite (NDVI)
# Objetivo: Enriquecer o dataset com o NDVI médio da safra para cada município/ano.
# ==============================================================================
# Célula 1: Setup e Carregamento de Dados (como já definimos).
# Célula 2: Adição das Geometrias (como já definimos).
# Célula 3: Definição da Função de Coleta de NDVI (como já definimos)
# Célula 4: CÉLULA DE TESTE (a nova que vamos criar agora).
# Célula 5: Execução do Loop Principal (a célula lenta).
    #============================================================================
# --- Importações de Bibliotecas ---
import pandas as pd
import geopandas as gpd
import ee
import io
import os
import time
from tqdm import tqdm

# --- Inicialização da API do Google Earth Engine ---
print("Inicializando a API do Google Earth Engine...")
try:
    # Lembre-se de ter seu ID de projeto correto aqui
    ee.Initialize(project='agrovision-gcp-project')
    print("API do GEE inicializada com sucesso!")
except Exception as e:
    print(f"ERRO ao inicializar a API do GEE: {e}")

# --- Carregamento dos Dados ---
# Carregamos o dataset que já contém os dados do IBGE e do Clima
CAMINHO_DADOS_COM_CLIMA = '../data/processed/dataset_completo_com_clima.csv'
try:
    df_com_clima = pd.read_csv(CAMINHO_DADOS_COM_CLIMA)
    print(f"\nDataFrame com dados climáticos carregado. Total de registros: {len(df_com_clima)}")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{CAMINHO_DADOS_COM_CLIMA}' não foi encontrado.")
    print("Por favor, execute o notebook de coleta de dados climáticos primeiro.")

Inicializando a API do Google Earth Engine...
API do GEE inicializada com sucesso!

DataFrame com dados climáticos carregado. Total de registros: 3026


In [2]:
# --- ENRIQUECIMENTO GEOESPACIAL: Adicionando Geometrias ---

print("Iniciando o enriquecimento com as geometrias dos municípios...")

url_geometrias = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-100-mun.json'
gdf_municipios = gpd.read_file(url_geometrias)
print("Geometrias carregadas com sucesso.")

gdf_geometrias_essencial = gdf_municipios[['id', 'geometry']].copy()
gdf_geometrias_essencial.rename(columns={'id': 'cod_municipio'}, inplace=True)
gdf_geometrias_essencial['cod_municipio'] = pd.to_numeric(gdf_geometrias_essencial['cod_municipio'])

# Unificar com o DataFrame da célula anterior
df_geo = pd.merge(
    df_com_clima,
    gdf_geometrias_essencial,
    on='cod_municipio',
    how='left'
)

# Converter o DataFrame resultante em um GeoDataFrame
gdf_final = gpd.GeoDataFrame(df_geo, geometry='geometry')

print("\nSucesso! DataFrame convertido em um GeoDataFrame pronto para o GEE.")
gdf_final.info()

Iniciando o enriquecimento com as geometrias dos municípios...
Geometrias carregadas com sucesso.

Sucesso! DataFrame convertido em um GeoDataFrame pronto para o GEE.
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3026 entries, 0 to 3025
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   cod_municipio             3026 non-null   int64   
 1   municipio_nome            3026 non-null   object  
 2   uf                        3026 non-null   object  
 3   ano                       3026 non-null   int64   
 4   area_plantada_ha          3026 non-null   float64 
 5   quantidade_produzida_ton  3025 non-null   float64 
 6   rendimento_medio_kg_ha    3025 non-null   float64 
 7   latitude                  3026 non-null   float64 
 8   longitude                 3026 non-null   float64 
 9   precipitacao_media_anual  3011 non-null   float64 
 10  temp_max_media_anual      3011 non-null  

In [7]:
# --- DEFINIÇÃO DA FUNÇÃO DE COLETA DE NDVI (VERSÃO COMUNICATIVA) ---
import signal
def buscar_ndvi_para_geometria(linha_geodataframe):
    """
    Busca o NDVI com lógica adaptativa e INFORMA sobre o resultado de cada requisição.
    """
    municipio = linha_geodataframe['municipio_nome']
    ano = int(linha_geodataframe['ano'])

    def handler(signum, frame):
        raise TimeoutError("O pedido ao GEE demorou mais de 3 minutos e foi cancelado.")

    # Define um alarme de 180 segundos (3 minutos)
    signal.signal(signal.SIGALRM, handler)
    signal.alarm(300)
    # ----------------------------------------------------
    
    
    try:
        geometria_gdf = linha_geodataframe['geometry']
        if geometria_gdf is None or geometria_gdf.is_empty:
            print(f"\n[AVISO] Geometria inválida para {municipio}/{ano}. Retornando None.")
            return None
            
        geometria_ee = ee.Geometry(geometria_gdf.__geo_interface__)
        data_inicio = f'{ano}-02-01'
        data_fim = f'{ano}-08-31'
        
        def calcular_ndvi(imagem):
            return imagem.normalizedDifference(['B8', 'B4']).rename('NDVI')

        def mascarar_nuvens_scl(imagem):
            scl = imagem.select('SCL')
            mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
            return imagem.updateMask(mask)

        colecao_imagens = (ee.ImageCollection('COPERNICUS/S2_SR')
                           .filterBounds(geometria_ee)
                           .filterDate(data_inicio, data_fim)
                           .map(mascarar_nuvens_scl)
                           .map(calcular_ndvi))
        
        if colecao_imagens.size().getInfo() == 0:
            print(f"\n[INFO] Nenhuma imagem sem nuvens encontrada para {municipio}/{ano}. Retornando None.")
            return None
        
        ndvi_maximo_imagem = colecao_imagens.select('NDVI').max()
        
        try:
            # Tentativa com alta resolução
            ndvi_stats = ndvi_maximo_imagem.reduceRegion(
                reducer=ee.Reducer.mean(), geometry=geometria_ee, scale=30, maxPixels=1e9)
            resultado = ndvi_stats.get('NDVI').getInfo()
            if resultado is not None:
                print(f"\n[SUCESSO] NDVI para {municipio}/{ano} (30m): {resultado:.4f}")
            else:
                 print(f"\n[INFO] NDVI nulo após processamento para {municipio}/{ano} (30m).")
            return resultado
            
        except ee.EEException as e:
            if 'memory limit' in str(e).lower():
                print(f"\n[AVISO] Erro de memória com 30m para {municipio}/{ano}. Tentando com 250m...")
                # Tentativa com baixa resolução
                ndvi_stats_fallback = ndvi_maximo_imagem.reduceRegion(
                    reducer=ee.Reducer.mean(), geometry=geometria_ee, scale=250, maxPixels=1e9)
                resultado = ndvi_stats_fallback.get('NDVI').getInfo()
                if resultado is not None:
                    print(f"\n[SUCESSO] NDVI para {municipio}/{ano} (250m): {resultado:.4f}")
                else:
                    print(f"\n[INFO] NDVI nulo após processamento para {municipio}/{ano} (250m).")
                return resultado
            else:
                raise e
       
    except Exception as e:
        print(f"\n[ERRO] Falha crítica ao processar {municipio}/{ano}. Retornando None. Erro: {e}\n")
        return None

print("Função comunicativa 'buscar_ndvi_para_geometria' definida com sucesso.")

Função comunicativa 'buscar_ndvi_para_geometria' definida com sucesso.


In [4]:
# ==============================================================================
# CÉLULA DE TESTE: Verificação End-to-End
# ==============================================================================

print("--- INICIANDO TESTE COMPLETO DO PIPELINE DE NDVI ---")

# Selecionar um registro de teste diferente, de um ano mais recente
try:
    # Vamos procurar por um município conhecido por alta produção e num ano recente
    linha_teste = gdf_final[
        (gdf_final['municipio_nome'] == 'Chapadão do Sul') & (gdf_final['ano'] == 2022)
    ].iloc[0]
    
    print(f"Registro de teste selecionado: {linha_teste['municipio_nome']} ({linha_teste['uf']}) para o ano de {linha_teste['ano']}")

    # Chamar a função principal para este único registro
    ndvi_resultado_teste = buscar_ndvi_para_geometria(linha_teste)

    # Verificar o resultado do teste
    print("\n--- Resultado do Teste ---")
    if ndvi_resultado_teste is not None:
        print(f"NDVI Máximo Médio da Safra: {ndvi_resultado_teste:.4f}")
        print("\n✅ SUCESSO! A função está a funcionar e encontrou dados válidos.")
    elif ndvi_resultado_teste is None:
        print("A função retornou 'None'. Isto é normal (nenhuma imagem encontrada) para este caso específico.")
    else:
        print("❌ FALHA! Ocorreu um erro inesperado.")

except (IndexError, KeyError):
    print("ERRO: Não foi possível selecionar a linha de teste. Verifique o nome do município ou se o DataFrame está carregado.")

--- INICIANDO TESTE COMPLETO DO PIPELINE DE NDVI ---
Registro de teste selecionado: Chapadão do Sul (MS) para o ano de 2022


/home/GALVAO/aplicacoes/back/python/AgroVision/venv/lib/python3.13/site-packages/ee/deprecation.py:209: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)



[SUCESSO] NDVI para Chapadão do Sul/2022 (30m): 0.5072

--- Resultado do Teste ---
NDVI Máximo Médio da Safra: 0.5072

✅ SUCESSO! A função está a funcionar e encontrou dados válidos.


In [ ]:
# ==============================================================================
# !! CÉLULA DE EXECUÇÃO FINAL E LENTA !!
# Rode esta célula para coletar TODOS os dados de NDVI.
# ==============================================================================

from tqdm import tqdm
import time
import os

# Apagar backup antigo para garantir um começo limpo
caminho_backup_ndvi = '../data/processed/backup_dados_ndvi.csv'
if os.path.exists(caminho_backup_ndvi):
    os.remove(caminho_backup_ndvi)
    print("Arquivo de backup antigo removido. Começando do zero.")

lista_ndvi = []

print(f"\nIniciando a coleta de dados de NDVI para {len(gdf_final)} registros...")

# O tqdm agora ficará mais "quieto", pois os prints da função darão o feedback
for indice, linha in tqdm(gdf_final.iterrows(), total=len(gdf_final), leave=False):
    ndvi = buscar_ndvi_para_geometria(linha)
    lista_ndvi.append(ndvi)
    time.sleep(1) # Delay para não sobrecarregar a API
    
    # Lógica de Backup
    if (indice + 1) % 50 == 0:
        print(f"\n--- SALVANDO BACKUP PARCIAL NO REGISTRO {indice + 1} ---")
        df_backup = gdf_final.iloc[:len(lista_ndvi)].copy()
        df_backup['ndvi_max_safra'] = lista_ndvi
        df_backup.drop('geometry', axis=1).to_csv(caminho_backup_ndvi, index=False)

# --- Finalização ---
print("\nColeta de dados de NDVI concluída!")

df_final_com_ndvi = gdf_final.copy()
df_final_com_ndvi['ndvi_max_safra'] = lista_ndvi

caminho_dataset_finalissimo = '../data/processed/dataset_final_completo.csv'
df_final_com_ndvi.drop('geometry', axis=1).to_csv(caminho_dataset_finalissimo, index=False)

print(f"\nDataset final e completo salvo com sucesso em: {caminho_dataset_finalissimo}")
df_final_com_ndvi.info()

Arquivo de backup antigo removido. Começando do zero.

Iniciando a coleta de dados de NDVI para 3026 registros...


  0%|          | 0/3026 [00:00<?, ?it/s]


[SUCESSO] NDVI para Água Clara/2021 (30m): 0.7139


  0%|          | 1/3026 [03:10<160:23:30, 190.88s/it]


[SUCESSO] NDVI para Água Clara/2022 (30m): 0.4590


...


[SUCESSO] NDVI para Brasília/2022 (30m): 0.4380


100%|█████████▉| 3025/3026 [53:32:50<02:59, 179.84s/it]


[SUCESSO] NDVI para Brasília/2023 (30m): 0.4651



Coleta de dados de NDVI concluída!

Dataset final e completo salvo com sucesso em: ../data/processed/dataset_final_completo.csv
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3026 entries, 0 to 3025
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   cod_municipio             3026 non-null   int64   
 1   municipio_nome            3026 non-null   object  
 2   uf                        3026 non-null   object  
 3   ano                       3026 non-null   int64   
 4   area_plantada_ha          3026 non-null   float64 
 5   quantidade_produzida_ton  3025 non-null   float64 
 6   rendimento_medio_kg_ha    3025 non-null   float64 
 7   latitude                  3026 non-null   float64 
 8   longitude                 3026 non-null   float64 
 9   precipitacao_media_anual  3011 non-null   float64 
 10  temp_max_media_anual      3011 non-null   float64 
 11  temp_min_media_anual   

: 